<a href="https://colab.research.google.com/github/zal-eun/spanish-translation-model/blob/main/spanish_translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 import

In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

스페인어 텍스트 샘플

In [ ]:
lines =pd.read_csv("C:\\Users\\82105\\Desktop\\spa.txt",names=['eng','spa','lic'],sep='\t')
del lines['lic']
#lines = lines[:100]
len(lines)

139013

총 139013 개의 샘플 중 60000개만 저장한다. 

In [ ]:
lines = lines.loc[:,'eng':'spa']
lines = lines[0:400000]

문장의 시작과 끝을 슬라이싱 하기 위해서 \t와 \n을 추가한다. 

In [ ]:
lines.spa = lines.spa.apply(lambda x : '\t '+ x + ' \n')
lines.sample(1)

,eng,spa
19400,Control yourselves.,\t \t Controlaos. \n \n


알파벳 기준의 단어집합(vocab)을 형성하기 위하여 각 영어 리스트와 상응하는 스페인어 리스트 문자에 있는 글자를 읽어 저장한다.

In [ ]:
eng_vocab = set()
for line in lines.eng: 
    for char in line: 
        eng_vocab.add(char)

spa_vocab = set()
for line in lines.spa:
    for char in line:
        spa_vocab.add(char)

In [ ]:
eng_vocab_size = len(eng_vocab)+1
spa_vocab_size = len(spa_vocab)+1
print('english 문장의 단어 집합 수:',eng_vocab_size)
print('spanish 문장의 단어 집합 수:',spa_vocab_size)

english 문장의 단어 집합 수: 82
spanish 문장의 단어 집합 수: 103


데이터 범위 내에서 영어는 82개의 문자, 스페인어는 총 103의 문자가 존재함을 알 수 있다. 

In [ ]:
eng_vocab = sorted(list(eng_vocab))
spa_vocab = sorted(list(spa_vocab))

:각 번호를 할당시켜 정렬하는 enumerate를 통해 인덱스를 부여한다.<br>
인덱스 첫 값이 0인 점에서 +1을 해준다

In [ ]:
eng_to_index = dict([(word, i+1) for i, word in enumerate(eng_vocab)])
spa_to_index = dict([(word, i+1) for i, word in enumerate(spa_vocab)])
print(eng_to_index)
print(spa_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '\xa0': 76, '°': 77, 'é': 78, '‘': 79, '’': 80, '€': 81}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '?': 28, 'A': 29, 'B': 30, 'C': 31, 'D'

영어에 대한 정수 인코딩 진행(line에 있는 각 문자 char에 대해 정수로 변환시킴)

In [ ]:
encoder_input = []


for line in lines.eng:
  encoded_line = []
  for char in line:
    encoded_line.append(eng_to_index[char])
  encoder_input.append(encoded_line)
print('영어 문장의 정수 인코딩 :',encoder_input[:5])

영어 문장의 정수 인코딩 : [[30, 64, 9], [30, 64, 9], [30, 64, 9], [30, 64, 9], [31, 58, 9]]


tar(스페인어)에 대한 정수 디코딩

In [ ]:
decoder_input = []
for line in lines.spa:
  encoded_line = []
  for char in line:
    encoded_line.append(spa_to_index[char])
  decoder_input.append(encoded_line)
print('스페인어 문장의 정수 인코딩 :',decoder_input[:5])

스페인어 문장의 정수 인코딩 : [[1, 3, 1, 3, 50, 59, 14, 3, 2, 3, 2], [1, 3, 1, 3, 50, 59, 74, 59, 14, 3, 2, 3, 2], [1, 3, 1, 3, 50, 55, 79, 55, 14, 3, 2, 3, 2], [1, 3, 1, 3, 50, 91, 79, 55, 73, 59, 14, 3, 2, 3, 2], [1, 3, 1, 3, 36, 69, 66, 55, 14, 3, 2, 3, 2]]


영어문장으로부터 스페인어 문장에 targeting한 값이  올바르게 수행되었는지 확인하기 위해서 실제값을 함께 정수 인코딩 한다. 

In [ ]:
decoder_target = []
for line in lines.spa:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(spa_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('실제값 정수 인코딩 :',decoder_target[:5])

실제값 정수 인코딩 : [[3, 1, 3, 50, 59, 14, 3, 2, 3, 2], [3, 1, 3, 50, 59, 74, 59, 14, 3, 2, 3, 2], [3, 1, 3, 50, 55, 79, 55, 14, 3, 2, 3, 2], [3, 1, 3, 50, 91, 79, 55, 73, 59, 14, 3, 2, 3, 2], [3, 1, 3, 36, 69, 66, 55, 14, 3, 2, 3, 2]]


padding 작업을 위한 최대 문장 길이 세기 <br>
padding 작업을 하는 이유:자연어 처리 과정에서 문장의 길이를 임의로 맞춤으로써 하나의 행렬로 인식해 수월한 작업이 가능하다. 


In [ ]:
max_eng_len = max([len(line) for line in lines.eng])
max_spa_len = max([len(line) for line in lines.spa])
print('영어 문장의 최대 길이 :',max_eng_len)
print('스페인어 문장의 최대 길이 :',max_spa_len)

영어 문장의 최대 길이 : 36
스페인어 문장의 최대 길이 : 95


따라서 영어 데이터 길이는 36, 스페인어 데이터 길이는 91에 맞추어 패딩을 진행한다. <br>
pad_sequences 함수를 통해 공란에 0이 대체 되도록 패딩 자동 진행한다. ('post' : 최대 길이에 맞춰줌)

In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_eng_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_spa_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_spa_len, padding='post')

In [ ]:
from tensorflow.keras.utils import to_categorical

원-핫 인코딩:표현하고 싶은 단어의 인덱스에 1을 부여하고, 다른 인덱스에는 0을 부여하여 단어를 표현한다. <br>케라스에서 지원하는 to_categorical을 통해서 원-핫 인코딩을 진행할 수 있다. 

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, eng_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
#인코더에 입력을 넣으면 내부 상태 리턴
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

state_h: lstm 은닉상태
state_c: lstm 셀 상태

In [ ]:
decoder_inputs = Input(shape=(None, spa_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)


decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(spa_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
1250/1250 [==============================] - 922s 731ms/step - loss: 0.6993 - val_loss: 0.7748
Epoch 2/40
1250/1250 [==============================] - 924s 739ms/step - loss: 0.4980 - val_loss: 0.6763
Epoch 3/40
1250/1250 [==============================] - 937s 750ms/step - loss: 0.4435 - val_loss: 0.6197
Epoch 4/40
1250/1250 [==============================] - 913s 730ms/step - loss: 0.4109 - val_loss: 0.5808
Epoch 5/40
1250/1250 [==============================] - 1030s 824ms/step - loss: 0.3856 - val_loss: 0.5556
Epoch 6/40
1250/1250 [==============================] - 857s 686ms/step - loss: 0.3668 - val_loss: 0.5341
Epoch 7/40
1250/1250 [==============================] - 643s 515ms/step - loss: 0.3501 - val_loss: 0.5162
Epoch 8/40
1250/1250 [==============================] - 654s 523ms/step - loss: 0.3373 - val_loss: 0.4973
Epoch 9/40
1250/1250 [==============================] - 726s 581ms/step - loss: 0.3270 - val_loss: 0.4846
Epoch 10/40
1250/1250 [======================

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_eng = dict((i, char) for char, i in eng_to_index.items())
index_to_spa = dict((i, char) for char, i in spa_to_index.items())

In [ ]:
for seq_index in [11,222,3333,44444]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.eng[seq_index])
  print('정답 문장:', lines.spa[seq_index][2:len(lines.spa[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 27ms/step
-----------------------------------
입력 문장: Who?
정답 문장: 	 ¿Quién? 
 
번역 문장: 	 ¡Qué verde! 
1/1 [==============================] - 0s 21ms/step
-----------------------------------
입력 문장: I froze.
정답 문장: 	 Me helé. 
 
번역 문장:  Me encanta la comida. 
1/1 [==============================] - 0s 21ms/step
-----------------------------------
입력 문장: He got angry.
정답 문장: 	 Él se enfadó. 
 
번역 문장:  Él no está aquí. 
1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장: I'm not afraid of death.
정답 문장: 	 No le temo a la muerte. 
 
번역 문장:  Nada me estoy muriendo de verdad. 


제작한 전처리 함수를 포함한 번역기로 업데이트

In [ ]:
import unicodedata

In [ ]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3

In [ ]:
def preprocess_sentence(sent):
    sent = to_ascii(sent.lower())
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    sent = re.sub(r"[^a-zA-Z!.?¿¡]+", r" ", sent)
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [ ]:
def to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [ ]:
#Ex
spa_sent = u'¡mañana, el árbol será más grande que hoy!'
print('전처리 전 스페인어 문장 :', spa_sent)
print('전처리 후 스페인어 문장 :', preprocess_sentence(spa_sent))


전처리 전 스페인어 문장 : ¡mañana, el árbol será más grande que hoy!
전처리 후 스페인어 문장 : ¡manana el arbol sera mas grande que hoy !


입력 시퀀스에는 시작을 의미하는 <sos> 토큰을, 출력 시퀀스에는 종료를 의미하는 <eos>토큰을 추가한다. 

In [ ]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("C:\\Users\\82105\\Desktop\\spa.txt", "r") as lines:
    for i, line in enumerate(lines):
      eng_line, spa_line, _ = line.strip().split('\t')

      
      eng_line = [w for w in preprocess_sentence(eng_line).split()]

     
      spa_line = preprocess_sentence(spa_line)
      spa_line_in = [w for w in ("<sos> " + spa_line).split()]
      spa_line_out = [w for w in (spa_line + " <eos>").split()]

      encoder_input.append(eng_line)
      decoder_input.append(spa_line_in)
      decoder_target.append(spa_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target